# Stiffness and Damping Matrices for Compressible Fluids

This notebook calculates the stiffness (K) and damping (C) matrices for compressible fluids such as air in journal bearings. The calculations account for the compressibility effects, including pressure-dependent viscosity and density.

In [17]:
import numpy as np

# Define parameters
L = 1.0  # Length of the bearing (half-length used in integration limits)
R = 1.0  # Radius of the bearing surface
mu = 1.0 # Viscosity
p0 = 1.0 # Reference pressure
omega = 1.0 # Angular frequency
perturbation_x = 0.01
perturbation_y = 0.01

# Discretization settings
num_theta = 100
num_z = 100
theta_vals = np.linspace(0, 2 * np.pi, num_theta)
z_vals = np.linspace(-L / 2, L / 2, num_z)

def compute_h(theta, z, delta_x, delta_y):
    return 1.0 + delta_x * np.cos(theta) + delta_y * np.sin(theta)

def compute_pressure_gradient(theta, z, h):
    # Here we should compute the actual pressure gradient using h, theta, z, and other relevant parameters.
    # This example assigns a simple distribution for illustrative purposes.
    return np.sin(theta) * np.cos(z) * h

def compute_velocity_shear(theta, z, h):
    # Computes velocity shear as a distribution; adapt this to your specific case
    return np.cos(theta) * np.sin(z) * h

def compute_delta_pressures(h, theta, z):
    pressure_gradient = compute_pressure_gradient(theta, z, h)
    velocity_shear = compute_velocity_shear(theta, z, h)

    delta_pk = -(pressure_gradient - velocity_shear) / (12 * mu * h**3)
    delta_pc = -omega * delta_pk  # Example phase relationship
    return delta_pk, delta_pc

def integrate_dynamic_pressure(delta_pressure_array, thetas, r, z_values, direction='x'):
    integral_value = 0.0
    for i, z in enumerate(z_values):
        for j, theta in enumerate(thetas):
            if direction == 'x':
                integral_value += delta_pressure_array[j, i] * np.cos(theta) * r * (2 * np.pi / num_theta) * (L / num_z)
            elif direction == 'y':
                integral_value += delta_pressure_array[j, i] * np.sin(theta) * r * (2 * np.pi / num_theta) * (L / num_z)
    return integral_value 

delta_pk = np.zeros((num_theta, num_z))
delta_pc = np.zeros((num_theta, num_z))

for i, theta in enumerate(theta_vals):
    for j, z in enumerate(z_vals):
        h = compute_h(theta, z, perturbation_x, perturbation_y)
        delta_pk[i, j], delta_pc[i, j] = compute_delta_pressures(h, theta, z)

# Calculate stiffness coefficients
k_xx = -integrate_dynamic_pressure(delta_pk, theta_vals, R, z_vals, direction='x') / perturbation_x
k_yx = -integrate_dynamic_pressure(delta_pk, theta_vals, R, z_vals, direction='y') / perturbation_x
k_xy = -integrate_dynamic_pressure(delta_pk, theta_vals, R, z_vals, direction='x') / perturbation_y
k_yy = -integrate_dynamic_pressure(delta_pk, theta_vals, R, z_vals, direction='y') / perturbation_y

# Calculate damping coefficients
c_xx = -integrate_dynamic_pressure(delta_pc, theta_vals, R, z_vals, direction='x') / perturbation_x
c_yx = -integrate_dynamic_pressure(delta_pc, theta_vals, R, z_vals, direction='y') / perturbation_x
c_xy = -integrate_dynamic_pressure(delta_pc, theta_vals, R, z_vals, direction='x') / perturbation_y
c_yy = -integrate_dynamic_pressure(delta_pc, theta_vals, R, z_vals, direction='y') / perturbation_y

# Print results
print(f"Stiffness matrix k_ij: \n[[{k_xx}, {k_xy}]\n [{k_yx}, {k_yy}]]")
print(f"Damping matrix c_ij: \n[[{c_xx}, {c_xy}]\n [{c_yx}, {c_yy}]]")



Stiffness matrix k_ij: 
[[0.0037257963540085807, 0.0037257963540085807]
 [24.83781437080759, 24.83781437080759]]
Damping matrix c_ij: 
[[-0.0037257963540085807, -0.0037257963540085807]
 [-24.83781437080759, -24.83781437080759]]
